In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

In [ ]:
!git clone https://github.com/MadisonAcosta/PyTutor.git


In [ ]:
import json
from collections import deque
import random

# Cargar intents y evaluaciones para el chat
with open("intenstss.json", "r", encoding="utf-8") as file:
    intents = json.load(file)

# Cargar datos de celulares
with open("celulares_con_benchmarks.json", "r", encoding="utf-8") as file:
    celulares = json.load(file)

# Memoria del chatbot
memoria_chat = deque(maxlen=5)
preferencias_usuario = {}
inicio_recoleccion = False

# Extraer preguntas del flujo desde intents.json
preguntas_flujo = intents["evaluacionesChat"][0]["responses"]
indice_pregunta = 0

# Extraer respuestas de saludos
saludos_patterns = {pat.lower() for intent in intents["intents"] if intent["tag"] == "saludo" for pat in intent["patterns"]}
saludos_responses = [resp for intent in intents["intents"] if intent["tag"] == "saludo" for resp in intent["responses"]]

def recomendar_celular():
    celulares_filtrados = celulares
    for clave, valor in preferencias_usuario.items():
        if clave == "presupuesto":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("Launched Price (USA)", float("inf")) <= valor]
        elif clave == "camara":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("Front Camera (MP)", 0) >= valor]
        elif clave == "rendimiento":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("RAM (GB)", 0) >= valor]
        elif clave == "bateria":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("Battery Capacity (mAh)", 0) >= valor]
        elif clave == "marca":
            celulares_filtrados = [c for c in celulares_filtrados if valor.lower() in c.get("Company Name", "").lower()]

    return celulares_filtrados[0] if celulares_filtrados else None

respuestas_positivas = {pat.lower() for intent in intents["evaluacionesChat"] if intent["tag"] == "respuestas_positivas" for pat in intent["responses"]}
variantes_celulares = {pat.lower() for intent in intents["evaluacionesChat"] if intent["tag"] == "variantes_celulares" for pat in intent["responses"]}
variantes_recomendar = {pat.lower() for intent in intents["evaluacionesChat"] if intent["tag"] == "variantes_recomendar" for pat in intent.get("variantes", [])}

def chatbot():
    global inicio_recoleccion, preferencias_usuario, indice_pregunta
    print("Hola, ¿en qué puedo ayudarte?")
    while True:
        user_input = input("Tú: ").strip().lower()

        if not user_input:
            print("Chatbot: No entendí tu respuesta. Por favor, escribe algo.")
            continue

        if user_input == "salir":
            print("Chatbot: ¡Hasta luego!")
            break

        # Verificar si es un saludo antes de cualquier otra respuesta
        if user_input in saludos_patterns:
            print(f"Chatbot: {random.choice(saludos_responses)}")
            continue

        if not inicio_recoleccion:
            if any(word in user_input.lower() for word in (variantes_recomendar | variantes_celulares | {"jugar", "busco"})):
                print("Chatbot: Claro, dime tus preferencias.")
                inicio_recoleccion = True
                indice_pregunta = 0
                preferencias_usuario.clear()
            else:
                print("Chatbot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?")
            continue

        # Capturar respuestas del usuario y avanzar en el flujo
        if indice_pregunta < len(preguntas_flujo):
            pregunta_actual = preguntas_flujo[indice_pregunta]["pregunta"]

            if "batería" in pregunta_actual.lower() and user_input.lower() in respuestas_positivas:
                preferencias_usuario["bateria"] = 4000
            elif "presupuesto" in pregunta_actual.lower() and user_input.isdigit():
                preferencias_usuario["presupuesto"] = int(user_input)
            elif "marca" in pregunta_actual.lower():
                preferencias_usuario["marca"] = user_input
            elif "cámara" in pregunta_actual.lower() and user_input.lower() in respuestas_positivas:
                preferencias_usuario["camara"] = 12
            elif "rendimiento" in pregunta_actual.lower() and user_input.lower() in respuestas_positivas:
                preferencias_usuario["rendimiento"] = 8


            indice_pregunta += 1
            if indice_pregunta < len(preguntas_flujo):
                print(f"Chatbot: {preguntas_flujo[indice_pregunta]['pregunta']}")
            else:
                celular_recomendado = recomendar_celular()
                if celular_recomendado:
                    print(f"Chatbot: Te recomiendo el {celular_recomendado['Model Name']} de {celular_recomendado['Company Name']}.\n"
                          f"Precio: ${celular_recomendado.get('Launched Price (USA)', 'Desconocido')}\n"
                          f"Batería: {celular_recomendado.get('Battery Capacity (mAh)', 'N/A')} mAh\n"
                          f"Cámara Frontal: {celular_recomendado.get('Front Camera (MP)', 'N/A')} MP\n"
                          f"RAM: {celular_recomendado.get('RAM (GB)', 'N/A')} GB\n"
                          f"Pantalla: {celular_recomendado.get('Screen Size (inches)', 'N/A')} pulgadas")
                else:
                    print("Chatbot: No encontré un celular que cumpla con todas tus preferencias. Intenta ajustar algunos criterios.")
                inicio_recoleccion = False
                indice_pregunta = 0
                preferencias_usuario.clear()

chatbot()


Hola, ¿en qué puedo ayudarte?


Chatbot: Claro, dime tus preferencias.
Chatbot: ¿Cuánto es tu presupuesto máximo? (Ingresa un número)
Chatbot: ¿Qué marca prefieres? (Escribe una marca específica)
Chatbot: ¿Quieres una buena cámara? (Sí/No)
Chatbot: ¿Necesitas un celular con buen rendimiento? (Sí/No)
Chatbot: No encontré un celular que cumpla con todas tus preferencias. Intenta ajustar algunos criterios.
Chatbot: No entendí tu respuesta. Por favor, escribe algo.
Chatbot: No entendí tu respuesta. Por favor, escribe algo.
Chatbot: Claro, dime tus preferencias.
Chatbot: ¿Cuánto es tu presupuesto máximo? (Ingresa un número)
Chatbot: No entendí tu respuesta. Por favor, escribe algo.
Chatbot: ¿Qué marca prefieres? (Escribe una marca específica)
Chatbot: ¿Quieres una buena cámara? (Sí/No)
Chatbot: ¿Necesitas un celular con buen rendimiento? (Sí/No)
Chatbot: No encontré un celular que cumpla con todas tus preferencias. Intenta ajustar algunos criterios.
Chatbot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?
Chatbot